In [1]:
import pandas as pd
# from google.colab import drive
# import warnings
# warnings.filterwarnings("ignore")
import sklearn
from tqdm import tqdm
import networkx as nx
from scipy.spatial.distance import cdist
import matplotlib.pyplot as plt
from sklearn.cluster import SpectralClustering
from sklearn.cluster import KMeans
from sklearn import preprocessing as p
import math
import numpy as np
import collections
from scipy.spatial import Voronoi, voronoi_plot_2d
# from shapely.geometry import Polygon
import scipy as sp
import scipy.spatial
import sys
import matplotlib.image as mpimg
import sklearn.metrics as metrics
from sklearn.calibration import calibration_curve
from spektral.data import Dataset, Graph, DisjointLoader
from spektral.layers import CrystalConv, GlobalAvgPool
from tensorflow.keras.layers import Dense, Input, Dropout
from tensorflow.keras.losses import BinaryCrossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.metrics import AUC

In [2]:
folder = r'C:\Users\Mahender\Documents\MS docs\Final docs\Rowan University\Research_project'

In [3]:
def data_cleaning(data):
    column_names = data.columns.tolist()
    if column_names[3] == 'Player11':
        data = data.rename(columns={'Player11': '11_x', 'Unnamed: 4': '11_y', 'Player1': '1_x',
                                    'Unnamed: 6': '1_y', 'Player2': '2_x', 'Unnamed: 8': '2_y', 'Player3': '3_x',
                                    'Unnamed: 10': '3_y',
                                    'Player4': '4_x', 'Unnamed: 12': '4_y', 'Player5': '5_x', 'Unnamed: 14': '5_y',
                                    'Player6': '6_x',
                                    'Unnamed: 16': '6_y', 'Player7': '7_x', 'Unnamed: 18': '7_y', 'Player8': '8_x',
                                    'Unnamed: 20': '8_y',
                                    'Player9': '9_x', 'Unnamed: 22': '9_y', 'Player10': '10_x', 'Unnamed: 24': '10_y',
                                    'Player12': '12_x',
                                    'Unnamed: 26': '12_y', 'Player13': '13_x', 'Unnamed: 28': '13_y',
                                    'Player14': '14_x', 'Unnamed: 30': '14_y',
                                    'Ball': 'Ball_x', 'Unnamed: 32': 'Ball_y'}).copy()
        data.loc[95703:, '4_x'] = data.loc[95703:, '12_x']
        data.loc[95703:, '4_y'] = data.loc[95703:, '12_y']
        data.loc[106111:, '8_x'] = data.loc[106111:, '13_x']
        data.loc[106111:, '8_y'] = data.loc[106111:, '13_y']
        data.loc[125738:, '10_x'] = data.loc[125738:, '14_x']
        data.loc[125738:, '10_y'] = data.loc[125738:, '14_y']

        data.to_csv('outputaw_data.csv', header=True)

        data = data.drop(['12_x', '12_y', '13_x', '13_y', '14_x', '14_y'], axis=1).copy()
        data = data.reindex(
            ['Period', 'Time [s]', '1_x', '1_y', '2_x', '2_y', '3_x', '3_y', '4_x', '4_y', '5_x', '5_y', '6_x', '6_y',
             '7_x', '7_y', '8_x',
             '8_y', '9_x', '9_y', '10_x', '10_y', '11_x', '11_y', 'Ball_x', 'Ball_y', 'Frame'], axis=1)

    else:

        data = data.rename(columns={'Player25': '25_x', 'Unnamed: 4': '25_y', 'Player15': '15_x',
                                    'Unnamed: 6': '15_y', 'Player16': '16_x', 'Unnamed: 8': '16_y',
                                    'Player17': '17_x', 'Unnamed: 10': '17_y',
                                    'Player18': '18_x', 'Unnamed: 12': '18_y', 'Player19': '19_x',
                                    'Unnamed: 14': '19_y', 'Player20': '20_x',
                                    'Unnamed: 16': '20_y', 'Player21': '21_x', 'Unnamed: 18': '21_y',
                                    'Player22': '22_x', 'Unnamed: 20': '22_y',
                                    'Player23': '23_x', 'Unnamed: 22': '23_y', 'Player24': '24_x',
                                    'Unnamed: 24': '24_y', 'Ball': 'Ball_x', 'Unnamed: 28': 'Ball_y'
                                    }).copy()

        data.loc[87985:, '22_x'] = data.loc[87985:, 'Player 26']
        data.loc[87985:, '22_y'] = data.loc[87985:, 'Unnamed: 26']

        data = data.drop(['Player 26', 'Unnamed: 26'
                          ], axis=1).copy()
        data.to_csv('outputho_data.csv', header=True)

        data = data.reindex(
            ['Frame', '15_x', '15_y', '16_x', '16_y', '17_x', '17_y', '18_x', '18_y', '19_x', '19_y', '20_x', '20_y',
             '21_x', '21_y', '22_x', '22_y', '23_x', '23_y', '24_x', '24_y', '25_x', '25_y', 'Ball_x', 'Ball_y'],
            axis=1)

    # data = data.head(500)
    data = data.dropna(subset=['Ball_x', 'Ball_y'])
    data = data.set_index('Frame').copy()
    return data
data_h = pd.read_csv(folder + '\\' + r'Sample_data\sample-data\data\Sample_Game_2\Sample_Game_2_RawTrackingData_Home_Team.csv', skiprows=2)
# print('h done')
data_a = pd.read_csv(folder + '\\' + r'Sample_data\sample-data\data\Sample_Game_2\Sample_Game_2_RawTrackingData_Away_Team.csv', skiprows=2, low_memory=False)
events = pd.read_csv(folder + '\\' + r'Sample_data\sample-data\data\Sample_Game_2\Sample_Game_2_RawEventsData.csv')
df1 = data_cleaning(data_h)
df2 = data_cleaning(data_a)
df1 = df1.drop(['Ball_x', 'Ball_y'], axis=1).copy()
data = pd.merge(df1, df2, left_index=True, right_index=True)

In [4]:
print(data.isna().sum())

Period      0
Time [s]    0
1_x         0
1_y         0
2_x         0
2_y         0
3_x         0
3_y         0
4_x         0
4_y         0
5_x         0
5_y         0
6_x         0
6_y         0
7_x         0
7_y         0
8_x         0
8_y         0
9_x         0
9_y         0
10_x        0
10_y        0
11_x        0
11_y        0
15_x        0
15_y        0
16_x        0
16_y        0
17_x        0
17_y        0
18_x        0
18_y        0
19_x        0
19_y        0
20_x        0
20_y        0
21_x        0
21_y        0
22_x        0
22_y        0
23_x        0
23_y        0
24_x        0
24_y        0
25_x        0
25_y        0
Ball_x      0
Ball_y      0
dtype: int64


In [5]:
data['Time [s]']=data['Time [s]'].astype('int64')

# Dropping the duplicates based on time and keeping 1st record of each sec
data.drop_duplicates(subset = ['Time [s]'], keep = 'first', inplace = True)

# Denormalizing the coordinates
for i in data.columns:
  if 'x' in i:
    data[i]=data[i]*105
  elif 'y' in i:
    data[i]=data[i]*68

In [6]:
# separating first and second half of the game

first_half=data[data['Period']==1]
second_half=data[data['Period']==2]

# just a way to get coords of a player throughout the game
coord=list(zip(data['1_x'].tolist(),data['1_y'].tolist()))

In [7]:
# zipping coodinates of players from first half in points_22 in the format frame: [(x,y),....]
# points_22 shape is 1987x22x2

players_id=[1,2,3,4,5,6,7,8,9,10,11,15,16,17,18,19,20,21,22,23,24,25]
points_22={}
for count,i in enumerate(first_half.index.tolist()):
  x,y=[],[]
  for j in players_id:
    x.append((first_half[f'{j}_x'].tolist())[count])
    y.append((first_half[f'{j}_y'].tolist())[count])
  point=zip(x,y)
  points_22[i]=list(point)

points_22_final={}
for count,i in enumerate(second_half.index.tolist()):
  x_2,y_2=[],[]
  for j in players_id:
    x_2.append((second_half[f'{j}_x'].tolist())[count])
    y_2.append((second_half[f'{j}_y'].tolist())[count])
  point=zip(x_2,y_2)
  points_22_final[i]=list(point)

ball_first=list(zip(first_half['Ball_x'].values.tolist(),first_half['Ball_y'].values.tolist()))

ball_second=list(zip(second_half['Ball_x'].values.tolist(),second_half['Ball_y'].values.tolist()))

In [30]:
first_half.index

Int64Index([   51,    75,   100,   125,   150,   175,   200,   225,   250,
              275,
            ...
            67625, 67650, 67675, 67700, 67725, 67750, 67775, 67800, 67825,
            67850],
           dtype='int64', name='Frame', length=1987)

In [8]:
# instant player data captures the player_id, nearest points count from knn, area for each frame
# "Player", i+1, "Nearest Points Count:", count, "Area : ", (count/(68*105)*1000)
# player_count_area_1 is for all frames

import numpy as np
from sklearn.neighbors import NearestNeighbors

# Generate 1000 random points on the soccer field
random_points = np.random.uniform(low=[0, 0], high=[105, 68], size=(10000, 2))

player_count_area_1 = []

# Define the coordinates of the 22 players on the soccer field
for i,j in points_22.items():
  player_coordinates = points_22[i]


# Create a NearestNeighbors object and fit it with player coordinates
  knn = NearestNeighbors(n_neighbors=1)
  knn.fit(player_coordinates)

# Find the nearest neighbor for each random point
  distances, indices = knn.kneighbors(random_points)

# Count the number of nearest points for each player
  nearest_points_count = np.zeros(22)
  for index in indices.flatten():
      player_index = index  # Assuming the player indices start from 0
      nearest_points_count[player_index] += 1

  instant_player_data = []


# Print the nearest points count for each player
  for i, count in enumerate(nearest_points_count):
    # print("Player", i+1, "Nearest Points Count:", count, "Area : ", (count/(68*105)*1000))
    instant_player_data.append((i+1,int(count), count/(68*105)*1000))
  player_count_area_1.append(instant_player_data)

In [9]:
random_points = np.random.uniform(low=[0, 0], high=[105, 68], size=(10000, 2))

player_count_area_2 = []

# Define the coordinates of the 22 players on the soccer field
for i,j in points_22_final.items():
  player_coordinates = points_22_final[i]


# Create a NearestNeighbors object and fit it with player coordinates
  knn = NearestNeighbors(n_neighbors=1)
  knn.fit(player_coordinates)

# Find the nearest neighbor for each random point
  distances, indices = knn.kneighbors(random_points)

# Count the number of nearest points for each player
  nearest_points_count = np.zeros(22)
  for index in indices.flatten():
      player_index = index  # Assuming the player indices start from 0
      nearest_points_count[player_index] += 1

  instant_player_data = []


# Print the nearest points count for each player
  for i, count in enumerate(nearest_points_count):
    # print("Player", i+1, "Nearest Points Count:", count, "Area : ", (count/(68*105)*1000))
    instant_player_data.append((i+1,int(count), count/(68*105)*1000))
  player_count_area_2.append(instant_player_data)

In [10]:
# influence is calculated from "Area : ", (count/(68*105)*1000) where is count is number of nearest points from KNN for all frames
# influence format is [[influence of player 1, 2,...],[influence of player 1, 2,...],...]
# influence_1half shape is 1987*22*1

influence_1half = []

for i in range(len(player_count_area_1)):
  influence_at_i = [t[2] for t in player_count_area_1[i]]
  influence_1half.append(influence_at_i)

In [11]:
influence_2half = []

for i in range(len(player_count_area_2)):
  influence_at_i = [t[2] for t in player_count_area_2[i]]
  influence_2half.append(influence_at_i)

In [12]:
# Space occupancy is for two teams for first half is: SO_A, SO_B
# SO_A format is {(frame: Space occupancy)}
# SOA_1_zer fills zeroes in frames from 2 to 2714, since size of SO_A is 1987 and max of SO_A is 2714
# SO_A shape is 1987x1
# SO_A[frame_i] + SO_B[frame_i] = 1

indx = first_half.index.tolist()
#print(indx)
SO_A,SO_B={},{}
for coun,indexx in enumerate(indx):
 # print(coun)
  dist_1=[]
  tim = first_half.loc[indexx]['Time [s]']
  ball = ball_first[coun]
  points = points_22[indexx]
#   if coun==4:
#     print(points)
  for i in points:
    eucli = math.dist(i, ball)

    if eucli!=0:
      eucli = (1.00000000 / eucli)
      dist_1.append(eucli)
    elif eucli==0:
      eucli = 1  # if distance less than 1 meter then weightage is 1
      dist_1.append(eucli)
  IP=influence_1half[coun]


  Iteam_A = np.dot(np.array(dist_1[:11]),np.array(IP[:11]))
  Iteam_B = np.dot(np.array(dist_1[11:]),np.array(IP[11:]))

  w_a = 1/math.dist(ball, (0,68/2))
  w_b = 1/math.dist(ball, (105,68/2))
  SO_A[tim]= (w_a*Iteam_A)/((w_a*Iteam_A) + (w_b*Iteam_B))
  SO_B[tim] = (w_b*Iteam_B)/((w_a*Iteam_A) + (w_b*Iteam_B))
# print(SO_A,SO_B)
SOA_1_zer,SOB_1_zer={},{}
for i in range(2,2715,1):
  if  i in SO_A.keys():
    SOA_1_zer[i]=SO_A[i]
    SOB_1_zer[i]=SO_B[i]
  else:
    SOA_1_zer[i]=0
    SOB_1_zer[i]=0

In [13]:
# Space occupancy is for two teams for second half is: SO_A_2, SO_B_2
# SO_A_2 shape is 1458x1

indx = second_half.index.tolist()
#print(indx)
SO_A_2,SO_B_2={},{}
for coun,indexx in enumerate(indx):
 # print(coun)
  dist_1=[]
  tim = second_half.loc[indexx]['Time [s]']
  ball = ball_second[coun]
  points = points_22_final[indexx]
#   if coun==4:
#     print(points)
  for i in points:
    eucli = math.dist(i, ball)

    if eucli!=0:
      eucli = (1.00000000 / eucli)
      dist_1.append(eucli)
    elif eucli==0:
      eucli = 1  # if distance less than 1 meter then weightage is 1
      dist_1.append(eucli)
  IP=influence_2half[coun]


  Iteam_A = np.dot(np.array(dist_1[:11]),np.array(IP[:11]))
  Iteam_B = np.dot(np.array(dist_1[11:]),np.array(IP[11:]))

  w_a = 1/math.dist(ball, (105,68/2))
  w_b = 1/math.dist(ball, (0,68/2))
  SO_A_2[tim]= (w_a*Iteam_A)/((w_a*Iteam_A) + (w_b*Iteam_B))
  SO_B_2[tim] = (w_b*Iteam_B)/((w_a*Iteam_A) + (w_b*Iteam_B))
# print(SO_A_2,SO_B_2)
SOA_2_zer,SOB_2_zer={},{}
for i in range(2719,5612,1):
  if  i in SO_A_2.keys():
    SOA_2_zer[i]=SO_A_2[i]
    SOB_2_zer[i]=SO_B_2[i]
  else:
    SOA_2_zer[i]=0
    SOB_2_zer[i]=0

### SO input for ASTGCN ###

In [14]:
# converting Space occupancy values to dataframe, then to array and then saving as npz

SO_df = pd.DataFrame(list(zip(SO_A.values(), SO_B.values())), index=[int(i) for i in SO_A.keys()])
SO2_df = pd.DataFrame(list(zip(SO_A_2.values(), SO_B_2.values())), index=[int(i) for i in SO_A_2.keys()])
SO_combined = pd.concat([SO_df, SO2_df])
# Convert the DataFrame to a NumPy array
SO_combined_array = SO_combined.to_numpy()

# Save the array to a .npz file with a key
np.savez_compressed(r'C:\Users\Mahender\Documents\MS docs\Final docs\Rowan University\Research_project\ASTGCN\Space_occupancy.npz',
                   data=SO_combined_array)

### Influence input for ASTGCN ###

In [15]:
# storing influence of first and second half in an npz file

influence_1half_array = np.array(influence_1half)
influence_2half_array = np.array(influence_2half)
combined_influence = np.vstack((influence_1half_array, influence_2half_array))

np.savez_compressed(r'C:\Users\Mahender\Documents\MS docs\Final docs\Rowan University\Research_project\ASTGCN\Influence.npz',
                   data=combined_influence)

In [16]:
player_coords = np.vstack((np.array(list(points_22.values())), np.array(list(points_22_final.values()))))
# Expanding dimensions of combined_influence to make it (3445, 22, 1)
expanded_influence = np.expand_dims(combined_influence, axis=-1)

# Concatenating along the third dimension to make the new array (3445, 22, 3)
coords_inf_array = np.concatenate((expanded_influence, player_coords), axis=-1)

# Now combined_array.shape should be (3445, 22, 3)
print(coords_inf_array.shape)

np.savez_compressed(r'C:\Users\Mahender\Documents\MS docs\Final docs\Rowan University\Research_project\ASTGCN\Coords_Influence.npz',
                   data=coords_inf_array)
np.save(r'C:\Users\Mahender\Documents\MS docs\Final docs\Rowan University\Research_project\ASTGCN\Coords_Influence.npy', coords_inf_array)

(3445, 22, 3)


### Ball coords input ###

In [17]:
ball_coords = np.vstack((np.array(ball_first), np.array(ball_second)))

print(ball_coords.shape)

np.savez_compressed(r'C:\Users\Mahender\Documents\MS docs\Final docs\Rowan University\Research_project\ASTGCN\Ball_coords.npz',
                   data=ball_coords)

(3445, 2)


In [18]:
# input_1: adjacancy matrices
# adjacancy matrices of 22 players for all frames of first half, here points 22 is the coords of all the players in the first half
# adj_matrices_c shape is 1987x22x22

def adj_mat_c(z_c):
    adj_matrices_c = []
    dist_matrices_c = []
    for index, valu in z_c.items():
        distances = cdist(valu, valu, 'euclidean')
        dist_matrices_c.append(distances)
        adj_matrix_c = np.where(distances<15, 1, 0)
        np.fill_diagonal(adj_matrix_c, 0)
        adj_matrices_c.append(adj_matrix_c)
    return adj_matrices_c, dist_matrices_c
adj_matrices_c, dist_matrices_c =adj_mat_c(points_22)
adj_matrices_c2, dist_matrices_c2 =adj_mat_c(points_22_final)

### adj and dist matrices input for ASTGCN ###

In [19]:
# storing adjacancy and distance matrices in npzy files

adj_matrices_c_array = np.array(adj_matrices_c)
dist_matrices_c_array = np.array(dist_matrices_c)
adj_matrices_c2_array = np.array(adj_matrices_c2)
dist_matrices_c2_array = np.array(dist_matrices_c2)

combined_adj_matrices = np.vstack((adj_matrices_c_array, adj_matrices_c2_array))
combined_dist_matrices = np.vstack((dist_matrices_c_array, dist_matrices_c2_array))

np.save(r'C:\Users\Mahender\Documents\MS docs\Final docs\Rowan University\Research_project\ASTGCN\adj_matrices.npy', combined_adj_matrices)
np.save(r'C:\Users\Mahender\Documents\MS docs\Final docs\Rowan University\Research_project\ASTGCN\dist_matrices.npy', combined_dist_matrices)

In [20]:
np.save(r'C:\Users\Mahender\Documents\MS docs\Final docs\Rowan University\Research_project\ASTGCN\initial_adj_matrices.npy', combined_adj_matrices[0])
np.save(r'C:\Users\Mahender\Documents\MS docs\Final docs\Rowan University\Research_project\ASTGCN\initial_dist_matrices.npy', combined_dist_matrices[0])

In [21]:
# coords of players for all frames without frame values

nested_x_y = [[np.array(t) for t in sublist] for i,sublist in points_22.items()]
for i,j in enumerate(nested_x_y):
  nested_x_y[i]=np.array(j)

In [22]:
# scipy.sparse.csr_matrix is just a better way to store matrix for faster performance in matrix operations

import scipy.sparse as sp
adj_matrices=[]
for i,j in enumerate(adj_matrices_c):
  adj_matrices.append(sp.csr_matrix(j))

In [23]:
# Space occupancy without frame

SO_A[list(SO_A.keys())[0]]
SO_A_li=[i for j,i in SO_A.items()]

In [24]:
data={}
data['adj']=adj_matrices
data['x']=nested_x_y
data['x'] = [np.append(x,np.ones(shape=(len(x), 1)), axis=1
            ) for x in data['x']]

In [25]:
# creating edges

ed=[]
for i,j in points_22.items():
  for k in j:
    print(k)
    break
  break

len(points_22[33550])

# for i,j in points_22.items():
#   distances = cdist(j, j, 'euclidean')
#   np.fill_diagonal(distances, 0)
#   empty_array_e = np.empty((0,))
#   for p in distances:
#     for  pd in p:
#     #   empty_array = np.array([])
#     #   empty_array = np.append(empty_array, np.array([pd]))
#       empty_array_e = np.vstack((empty_array_e, np.array([pd])))


#   ed.append(empty_array_e)

for i,j in points_22.items():
  distances = cdist(j, j, 'euclidean')
  np.fill_diagonal(distances, -1)
  l=[]
  for row,p in enumerate(distances):
    for column, p_d in enumerate(p):
      if p_d<15 and p_d!=-1:
        li=[]
        li.append(p_d)
        l.append(li)

  ed.append(l)
for ind,i in enumerate(ed):

  for ik,k in enumerate(i):
    i[ik]=np.array(k)

  ed[ind]=np.array(i)
for i in ed:
  for j in i:
    if len(j)!=1:
      raise Exception

(68.02635, 18.38108)


In [26]:
data['e']=ed

In [27]:
# from spektral.data import Dataset, Graph

# class CounterDataset(Dataset):
#     def __init__(self,  **kwargs):
#         super().__init__(**kwargs)
#         self.data = kwargs['data']
#         self.SO_A = kwargs['SO_A']

#     def read(self):
#         data=self.data
#         graphs = []
#         for adj_matrix,x, e,label in zip(data['adj'],data['x'],data['e'], self.SO_A):
#             graph = Graph(x=x,a=adj_matrix, e=e, y=label)
#             graphs.append(graph)
#         return graphs

In [28]:
# dataset = CounterDataset(data=data,SO_A= SO_A_li)

In [29]:
# pip install torch_scatter torch_sparse torch_cluster torch_spline_conv -f https://data.pyg.org/whl/torch-2.0.0+cu118.html